# About the dataset
**Name:** This column contains the full name of the athlete participating in the Olympic Games.<br>
**Sex:** This column indicates the gender of the athlete. It has two unique values: "M" for male and "F" for female.<br>
**Age:** This column represents the age of the athlete at the time of the competition.<br>
**Team:** The name of the team or delegation that the athlete represents in Olympics.<br>
**NOC:** It contains the three-letter country code assigned by the National Olympic Committee (NOC).<br>
**Year:** Represents the year in which the Olympic Games took place.<br>
**Season:** Indicates whether the Olympic Games occurred in the "Summer" or "Winter" season. This distinction is important because different sports are played in each season.<br>
**City:** The host city where the Olympic event took place. This information can be useful for analyzing the impact of location and climate conditions on athlete performance.<br>
**Sport:** Represents the broad category of the sport in which the athlete competed (e.g., Athletics, Swimming, Gymnastics).<br>
**Event:** The specific event within a sport in which the athlete participated (e.g., "100m Sprint", "Long Jump").<br>
**Medal:** Indicates the type of medal won by the athlete. Possible values include "Gold", "Silver", "Bronze", or "NaN" if no medal was won.<br>
**Country:** This column represents the full country name corresponding to the NOC code.<br>
**Height:** The height of the athlete is in centimetres.<br>
**Weight:** The weight of the athlete in kilograms.<br>

# Import the Libraries and Load the Data

In [3]:
#Import the Libraries
import numpy as np
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import OrdinalEncoder,StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.svm import SVR
!pip install xgboost
import xgboost as xgb
from xgboost import XGBRegressor
import time

In [4]:
#Loading the file
df = pd.read_csv('Athletes_summer_games.csv')
df_athlete = pd.read_csv('Olympic_Athlete_Biography.csv')
df_region = pd.read_csv('Olympic_Country_Profiles.csv')

In [5]:
df.sample(1)

,Unnamed: 0,Name,Sex,Age,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
207206,252153,Sandro Ricardo Rodrigues Viana,M,35.0,Brazil,BRA,2012 Summer,2012,Summer,London,Athletics,Athletics Men's 200 metres,NaN


In [6]:
# Getting all column name in same format
df_region.rename(columns={'noc':'NOC','country':'Country'},inplace = True)

In [7]:
df_region.sample(1)

,NOC,Country
216,EUN,Unified Team


In [8]:
# Merging Country in main dataset
df = df.merge(df_region,on='NOC',how = 'left')

In [9]:
# dropping unnecessary column
df = df.drop(columns=['Unnamed: 0','Games'])

In [10]:
df.sample(1)

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal,Country
106369,"Alapurackal ""Mercy"" Kuttan-Mathews",F,27.0,India,IND,1988,Summer,Seoul,Athletics,Athletics Women's 400 metres,NaN,India


In [11]:
# dropping unnecessary column and Getting all column name in same format
df_athlete = df_athlete.drop(columns=['athlete_id','sex','born','country','country_noc','description','special_notes'])
df_athlete.rename(columns={'name':'Name','height':'Height','weight':'Weight'},inplace=True)
df_athlete.sample(1)

,Name,Height,Weight
146534,Jamtsyn Bor,189.0,90.0


In [12]:
# Merging Height & Weight in main dataset
df = df.merge(df_athlete,on='Name',how='left')

In [13]:
df.sample(3)

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal,Country,Height,Weight
87397,Kazuhiro Iwatani,M,24.0,Japan,JPN,1988,Summer,Seoul,Equestrianism,"Equestrianism Mixed Three-Day Event, Individual",NaN,Japan,173.0,68.0
160234,Vladimir Vladimirovich Portnoy,M,29.0,Soviet Union,URS,1960,Summer,Roma,Gymnastics,Gymnastics Men's Pommelled Horse,NaN,Soviet Union,NaN,NaN
132788,Ian Millar,M,37.0,Canada,CAN,1984,Summer,Los Angeles,Equestrianism,"Equestrianism Mixed Jumping, Team",NaN,Canada,185.0,76.0


# Understanding of the data

In [15]:
#Dimensions of the data
df.shape

(241723, 14)

In [16]:
#Information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241723 entries, 0 to 241722
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Name     241723 non-null  object 
 1   Sex      241723 non-null  object 
 2   Age      232357 non-null  float64
 3   Team     241723 non-null  object 
 4   NOC      241723 non-null  object 
 5   Year     241723 non-null  int64  
 6   Season   241723 non-null  object 
 7   City     241723 non-null  object 
 8   Sport    241723 non-null  object 
 9   Event    241723 non-null  object 
 10  Medal    37196 non-null   object 
 11  Country  241469 non-null  object 
 12  Height   68601 non-null   float64
 13  Weight   68601 non-null   float64
dtypes: float64(3), int64(1), object(10)
memory usage: 25.8+ MB


In [17]:
# Converted all country values into string
df['Country'] = df['Country'].astype(str)

In [18]:
#Information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241723 entries, 0 to 241722
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Name     241723 non-null  object 
 1   Sex      241723 non-null  object 
 2   Age      232357 non-null  float64
 3   Team     241723 non-null  object 
 4   NOC      241723 non-null  object 
 5   Year     241723 non-null  int64  
 6   Season   241723 non-null  object 
 7   City     241723 non-null  object 
 8   Sport    241723 non-null  object 
 9   Event    241723 non-null  object 
 10  Medal    37196 non-null   object 
 11  Country  241723 non-null  object 
 12  Height   68601 non-null   float64
 13  Weight   68601 non-null   float64
dtypes: float64(3), int64(1), object(10)
memory usage: 25.8+ MB


In [19]:
# Missing values
df.isnull().sum()

Name            0
Sex             0
Age          9366
Team            0
NOC             0
Year            0
Season          0
City            0
Sport           0
Event           0
Medal      204527
Country         0
Height     173122
Weight     173122
dtype: int64

In [20]:
# checking duplicates values.
df.duplicated().sum()

1765

In [21]:
# Now there is no duplicate values.
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [22]:
# Need to maintain consistancy in Country column
Country_Mapping={'ROC':'Russian Olympic Committee'}
df['Country'] = df['Country'].replace(Country_Mapping)

In [23]:
df

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal,Country,Height,Weight
0,A Dijiang,M,24.0,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,People's Republic of China,NaN,NaN
1,A Lamusi,M,23.0,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,People's Republic of China,170.0,60.0
2,Gunnar Nielsen Aaby,M,24.0,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN,NaN
3,Edgar Lindenau Aabye,M,34.0,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,NaN
4,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,Netherlands,NED,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,NaN,Netherlands,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241718,ZYKOVA Yulia,F,25.0,Russia,ROC,2020,Summer,Tokyo,Shooting,50m Rifle 3 Positions Women,Silver,Russian Olympic Committee,NaN,NaN
241719,ZYUZINA Ekaterina,F,24.0,Russia,ROC,2020,Summer,Tokyo,Sailing,Women's One Person Dinghy - Laser Radial,NaN,Russian Olympic Committee,NaN,NaN
241720,ZYUZINA Ekaterina,F,24.0,Russia,ROC,2020,Summer,Tokyo,Sailing,Women's One Person Dinghy - Laser Radial,NaN,Russian Olympic Committee,NaN,NaN
241721,ZYZANSKA Sylwia,F,24.0,Poland,POL,2020,Summer,Tokyo,Archery,Women's Individual,NaN,Poland,NaN,NaN


# Model Building

In [25]:
# Adding 'is_medal' column 
df['is_medal'] = np.where(df['Medal'].isnull(),0,1)

In [26]:
# dropping duplicates on the basis Name,Event,Team,Country
df.drop_duplicates(subset=['Name','Event','Team','Country'])

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal,Country,Height,Weight,is_medal
0,A Dijiang,M,24.0,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,People's Republic of China,NaN,NaN,0
1,A Lamusi,M,23.0,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,People's Republic of China,170.0,60.0,0
2,Gunnar Nielsen Aaby,M,24.0,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN,NaN,0
3,Edgar Lindenau Aabye,M,34.0,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,NaN,1
4,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,Netherlands,NED,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,NaN,Netherlands,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241716,ZWOLINSKA Klaudia,F,22.0,Poland,POL,2020,Summer,Tokyo,Canoe Slalom,Women's Kayak,NaN,Poland,NaN,NaN,0
241717,ZYKOVA Yulia,F,25.0,Russia,ROC,2020,Summer,Tokyo,Shooting,50m Rifle 3 Positions Women,Silver,Russian Olympic Committee,NaN,NaN,1
241719,ZYUZINA Ekaterina,F,24.0,Russia,ROC,2020,Summer,Tokyo,Sailing,Women's One Person Dinghy - Laser Radial,NaN,Russian Olympic Committee,NaN,NaN,0
241721,ZYZANSKA Sylwia,F,24.0,Poland,POL,2020,Summer,Tokyo,Archery,Women's Individual,NaN,Poland,NaN,NaN,0


**Interpretation:** This dataset has been preprocessed to include a binary target variable 'is_medal', which can be treated as a continuous output for linear regression. By dropping medal-specific columns and removing duplicate athlete-event entries, the dataset is streamlined for modeling how various features (e.g., age, height, weight, sport) influence the likelihood of winning a medal.

In [28]:
# Filling NaN Values to maintain consistancy
for col in ['Height', 'Weight', 'Age']:
    sport_avg = df.groupby('Sport')[col].transform('mean')  # Sport-wise mean
    overall_avg = df[col].mean()  # Overall mean (all sports)
    
    # If sport_avg is NaN (i.e., sport has no values), fill it with overall_avg
    df[col] = df[col].fillna(sport_avg).fillna(overall_avg)

**Interpretation:** To handle missing values in key numeric columns like Height, Weight and Age, sport-specific averages were used for imputation, ensuring contextual relevance. If sport-level data was unavailable, the overall average was used as a fallback, preserving data integrity while minimizing bias before applying the regression model.

In [30]:
# Groupbying & Aggregating on the basis of following columns
df_1 = df.groupby(['Year','Sport','Country','Sex']).agg({'Height':'mean','Weight':'mean','Age':'mean','is_medal':'count'}).reset_index()
df_1

,Year,Sport,Country,Sex,Height,Weight,Age,is_medal
0,1896,Athletics,Australia,M,176.208009,69.208994,22.000000,3
1,1896,Athletics,Denmark,M,176.208009,69.208994,26.250000,4
2,1896,Athletics,France,M,176.208009,69.208994,22.416364,12
3,1896,Athletics,Germany,M,167.645717,62.931784,23.142857,14
4,1896,Athletics,Great Britain,M,177.892579,73.893423,23.214026,7
...,...,...,...,...,...,...,...,...
33273,2020,Wrestling,Ukraine,F,171.919772,74.544279,25.800000,5
33274,2020,Wrestling,Ukraine,M,171.919772,74.544279,28.000000,5
33275,2020,Wrestling,United States,F,171.919772,74.544279,27.833333,6
33276,2020,Wrestling,United States,M,171.919772,74.544279,27.333333,9


**Interpretation:** This aggregation summarizes the dataset by calculating the average Height, Weight, and Age of athletes grouped by Year, Sport, Country, and Sex. Additionally, the is_medal count reflects the number of athlete entries per group, helping to identify participation trends and performance demographics across different dimensions.

In [32]:
# Making Keycolumn for better understanding
df_1['Country'] = df_1['Country'].str.strip()
df_1['Sex'] = df_1['Sex'].str.strip()
df_1['Sport'] = df_1['Sport'].str.strip()
df_1['Country'] = df_1['Country'].str.replace(" ","_")
df_1['Sex'] = df_1['Sex'].str.replace(" ","_")
df_1['Sport'] = df_1['Sport'].str.replace(" ","_")
df_1['Key_col'] = df_1['Country'].astype(str)+ "_" + df_1['Sport'].astype(str)+ "_"+ df_1["Sex"].astype(str)
df_1

,Year,Sport,Country,Sex,Height,Weight,Age,is_medal,Key_col
0,1896,Athletics,Australia,M,176.208009,69.208994,22.000000,3,Australia_Athletics_M
1,1896,Athletics,Denmark,M,176.208009,69.208994,26.250000,4,Denmark_Athletics_M
2,1896,Athletics,France,M,176.208009,69.208994,22.416364,12,France_Athletics_M
3,1896,Athletics,Germany,M,167.645717,62.931784,23.142857,14,Germany_Athletics_M
4,1896,Athletics,Great_Britain,M,177.892579,73.893423,23.214026,7,Great_Britain_Athletics_M
...,...,...,...,...,...,...,...,...,...
33273,2020,Wrestling,Ukraine,F,171.919772,74.544279,25.800000,5,Ukraine_Wrestling_F
33274,2020,Wrestling,Ukraine,M,171.919772,74.544279,28.000000,5,Ukraine_Wrestling_M
33275,2020,Wrestling,United_States,F,171.919772,74.544279,27.833333,6,United_States_Wrestling_F
33276,2020,Wrestling,United_States,M,171.919772,74.544279,27.333333,9,United_States_Wrestling_M


**Interpretation:** These steps are part of data preprocessing to ensure consistency and avoid errors during modeling or merging. By stripping whitespace and replacing spaces with underscores, the categorical fields become cleaner and more uniform. The creation of a Key_col combines Country, Sport, and Sex into a unique identifier useful for grouping, indexing, or joining datasets.

In [34]:
# Finding only those rows who have >=2 data in history
abc = pd.DataFrame(df_1.groupby('Key_col')['Height'].count().sort_values()).reset_index()
abc = abc.rename(columns = {'Height':'number_of_rows'})
abc = abc[abc['number_of_rows']>=2]
abc

,Key_col,number_of_rows
2041,Italy_Boxing_F,2
2042,Kingdom_of_Saudi_Arabia_Fencing_M,2
2043,Kingdom_of_Saudi_Arabia_Judo_F,2
2044,Puerto_Rico_Gymnastics_F,2
2045,Romania_Volleyball_F,2
...,...,...
6288,Sweden_Athletics_M,29
6289,United_States_Swimming_M,29
6290,Great_Britain_Fencing_M,29
6291,France_Athletics_M,30


**Interpretation:** This code filters the dataset to retain only those Key_col groups (combinations of Country, Sport, and Sex) that have at least two data points. This ensures that there’s enough data within each group for meaningful statistical analysis or modeling, especially useful when applying group-based techniques like time-series or regression.

In [36]:
# Finding only those 'Key_col' who are in history and same available in the 2020 dataset 
intersection_list = list(set(list(df_1[df_1['Year'] == 2020]['Key_col'].unique())).intersection(set(abc['Key_col'].unique())))
len(intersection_list)

2195

**Interpretation:** This code identifies the intersection between Key_col groups present in the 2020 data and those that have sufficient data (≥2 rows) across all years. The resulting intersection_list contains valid and robust groups for analysis or modeling in 2020, ensuring consistency and reliability in predictions.

In [38]:
# Model Predictions
start_time = time.time()
final_df = pd.DataFrame()

# Loop over each key in intersection_list
for i in intersection_list:
    # Filter training data for the specific Key_col and Year <= 2016
    train_data_12 = df_1[(df_1['Key_col'] == i) & (df_1['Year'] <= 2016)].reset_index(drop=True)

    # Filter test data for Year 2020 and specific Key_col
    test_data_12 = df_1[(df_1['Year'] == 2020) & (df_1['Key_col'] == i)].reset_index(drop=True)

    # Skip iteration if there's no training or testing data (ignore silently)
    if train_data_12.empty or test_data_12.empty:
        continue  

    # Define Features and Target
    X_train = train_data_12[['Year', 'Height', 'Weight', 'Age']]
    y_train = train_data_12[['is_medal']]

    X_test = test_data_12[['Year', 'Height', 'Weight', 'Age']]
    y_test = test_data_12[['is_medal']]

    # Standardization & Normalization
    standard_scaler = StandardScaler()
    minmax_scaler = MinMaxScaler()

    # Fit scalers
    X_train_standard = standard_scaler.fit_transform(X_train)
    X_test_standard = standard_scaler.transform(X_test)

    X_train_minmax = minmax_scaler.fit_transform(X_train)
    X_test_minmax = minmax_scaler.transform(X_test)


    # Define Models
    models = {
        "Linear Regression": (LinearRegression(), X_train_standard, X_test_standard),
        "XGBoost": (XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42), X_train_standard, X_test_standard),
        "SVR": (SVR(kernel='rbf', C=1.0, epsilon=0.1), X_train_standard, X_test_standard),
        "Decision Tree": (DecisionTreeRegressor(), X_train_minmax, X_test_minmax),
        "Random Forest": (RandomForestRegressor(n_estimators=100, random_state=42, max_features='sqrt', n_jobs=-1), X_train_minmax, X_test_minmax),
    }

    # Store results for each model
    results = []
    for name, (model, X_train_scaled, X_test_scaled) in models.items():
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        results.append([i, name, y_pred[0], y_test.iloc[0, 0], mae, mse, rmse])

    # Append results to the final DataFrame
    results_df = pd.DataFrame(results, columns=["Key_col", "Model", "Predictions", "Actual", "MAE", "MSE", "RMSE"])
    final_df = pd.concat([final_df, results_df], ignore_index=True)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution Time: {execution_time:.4f} seconds")

Execution Time: 1055.2492 seconds


In [39]:
final_df

,Key_col,Model,Predictions,Actual,MAE,MSE,RMSE
0,Panama_Athletics_F,Linear Regression,[-0.5864202902629199],2,2.586420,6.689570e+00,2.586420
1,Panama_Athletics_F,XGBoost,1.003278,2,0.996722,9.934554e-01,0.996722
2,Panama_Athletics_F,SVR,2.322722,2,0.322722,1.041494e-01,0.322722
3,Panama_Athletics_F,Decision Tree,1.0,2,1.000000,1.000000e+00,1.000000
4,Panama_Athletics_F,Random Forest,1.96,2,0.040000,1.600000e-03,0.040000
...,...,...,...,...,...,...,...
10970,Tunisia_Fencing_M,Linear Regression,[0.6657071961910659],2,1.334293,1.780337e+00,1.334293
10971,Tunisia_Fencing_M,XGBoost,2.000341,2,0.000341,1.162391e-07,0.000341
10972,Tunisia_Fencing_M,SVR,2.242111,2,0.242111,5.861771e-02,0.242111
10973,Tunisia_Fencing_M,Decision Tree,2.0,2,0.000000,0.000000e+00,0.000000


**Interpretation:** This code evaluates the performance of multiple regression models (like Linear Regression, XGBoost, SVR, etc.) in predicting medal wins (is_medal) for the 2020 Olympics, using athlete features like Height, Weight, Age, and Year based on historical data (up to 2016). It applies standardization or normalization as needed and calculates key evaluation metrics (MAE, MSE, RMSE) for each Key_col group, compiling the predictions and actuals into a consolidated result DataFrame.

In [41]:
# Model Predictions with hyperparameter tuning

start_time = time.time()
final_df_1 = pd.DataFrame()

for i in intersection_list:
    train_data_12 = df_1[(df_1['Key_col'] == i) & (df_1['Year'] <= 2016)].reset_index(drop=True)
    test_data_12 = df_1[(df_1['Year'] == 2020) & (df_1['Key_col'] == i)].reset_index(drop=True)

    if train_data_12.empty or test_data_12.empty:
        continue  

    X_train = train_data_12[['Year', 'Height', 'Weight', 'Age']]
    y_train = train_data_12[['is_medal']]

    X_test = test_data_12[['Year', 'Height', 'Weight', 'Age']]
    y_test = test_data_12[['is_medal']]

    # Scaling
    standard_scaler = StandardScaler()
    minmax_scaler = MinMaxScaler()

    X_train_standard = standard_scaler.fit_transform(X_train)
    X_test_standard = standard_scaler.transform(X_test)

    X_train_minmax = minmax_scaler.fit_transform(X_train)
    X_test_minmax = minmax_scaler.transform(X_test)

     # Define & Tune Models (now inside the check block)
    models = {}

    if len(train_data_12) < 3:
        # Not enough data for cross-validation
        models["Ridge Regression"] = (Ridge(alpha=1.0), X_train_standard, X_test_standard)
        models["XGBoost"] = (XGBRegressor(objective='reg:squarederror', random_state=42), X_train_standard, X_test_standard)
        models["SVR"] = (SVR(kernel='rbf', C=1.0, epsilon=0.1), X_train_standard, X_test_standard)
    else:
        models["Ridge Regression"] = (Ridge(alpha=1.0), X_train_standard, X_test_standard)

        # XGBoost with basic tuning
        xgb_model = GridSearchCV(
            estimator=XGBRegressor(objective='reg:squarederror', random_state=42),
            param_grid={
                'n_estimators': [50, 100],
                'max_depth': [3, 5],
                'learning_rate': [0.05, 0.1]
            },
            scoring='neg_mean_squared_error',
            cv=3,
            n_jobs=-1
        )
        models["XGBoost"] = (xgb_model, X_train_standard, X_test_standard)

        # SVR with tuning
        svr_model = GridSearchCV(
            estimator=SVR(),
            param_grid={
                'C': [0.1, 1, 10],
                'epsilon': [0.01, 0.1],
                'kernel': ['rbf']
            },
            scoring='neg_mean_squared_error',
            cv=3,
            n_jobs=-1
        )
        models["SVR"] = (svr_model, X_train_standard, X_test_standard)

    # Decision Tree (no tuning)
    models["Decision Tree"] = (DecisionTreeRegressor(), X_train_minmax, X_test_minmax)

    # Random Forest (no tuning)
    models["Random Forest"] = (RandomForestRegressor(n_estimators=100, random_state=42, max_features='sqrt', n_jobs=-1), X_train_minmax, X_test_minmax)

    # Train and Evaluate
    results = []
    for name, (model, X_train_scaled, X_test_scaled) in models.items():
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)

        if hasattr(y_pred, '__len__'):
            pred_value = y_pred[0]
        else:
            pred_value = y_pred

        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        results.append([i, name, pred_value, y_test.iloc[0, 0], mae, mse, rmse])

    results_df = pd.DataFrame(results, columns=["Key_col", "Model", "Predictions", "Actual", "MAE", "MSE", "RMSE"])
    final_df_1 = pd.concat([final_df_1, results_df], ignore_index=True)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution Time: {execution_time:.4f} seconds")

Execution Time: 1982.9196 seconds


In [42]:
final_df_1

,Key_col,Model,Predictions,Actual,MAE,MSE,RMSE
0,Panama_Athletics_F,Ridge Regression,1.218950,2,0.781050,0.610039,0.781050
1,Panama_Athletics_F,XGBoost,1.596651,2,0.403349,0.162690,0.403349
2,Panama_Athletics_F,SVR,0.127279,2,1.872721,3.507083,1.872721
3,Panama_Athletics_F,Decision Tree,1.000000,2,1.000000,1.000000,1.000000
4,Panama_Athletics_F,Random Forest,1.960000,2,0.040000,0.001600,0.040000
...,...,...,...,...,...,...,...
10970,Tunisia_Fencing_M,Ridge Regression,3.039611,2,1.039611,1.080790,1.039611
10971,Tunisia_Fencing_M,XGBoost,2.120568,2,0.120568,0.014537,0.120568
10972,Tunisia_Fencing_M,SVR,2.024211,2,0.024211,0.000586,0.024211
10973,Tunisia_Fencing_M,Decision Tree,2.000000,2,0.000000,0.000000,0.000000


**Interpretation:** This code enhances model performance by applying basic hyperparameter tuning for XGBoost and SVR using GridSearchCV, while also adapting to limited data scenarios by conditionally skipping tuning when training samples are insufficient. It evaluates multiple regression models across different Key_col segments, optimizing for mean squared error and compiling predictions and metrics (MAE, MSE, RMSE) into a final result for the 2020 Olympics.

In [44]:
final_df.to_excel('Regression_results_no_tuning_final.xlsx',index=False)

In [45]:
final_df_1.to_excel('Regression_results_with_tuning_final.xlsx',index=False)